In [1]:
import requests

def baixar_dados_cemaden(estacao_id=7217, caminho_saida='data/raw/cemaden_7217.csv'):
    """
    Baixa os dados da estação CEMADEN de Suzano (ID 7217) e salva em um arquivo CSV.

    Args:
        estacao_id (int): ID da estação CEMADEN (default: 7217 - Suzano).
        caminho_saida (str): Caminho para salvar o arquivo CSV.
    """
    url = f'https://cemaden.gov.br/api/chuva/{estacao_id}/dados'  # Exemplo de endpoint, ajuste conforme necessário

    try:
        resposta = requests.get(url)
        resposta.raise_for_status()
        with open(caminho_saida, 'wb') as f:
            f.write(resposta.content)
        print(f'Dados salvos em {caminho_saida}')
    except Exception as e:
        print(f'Erro ao baixar dados: {e}')

# Exemplo de uso:
# baixar_dados_cemaden()

In [2]:
baixar_dados_cemaden()

Erro ao baixar dados: HTTPSConnectionPool(host='cemaden.gov.br', port=443): Max retries exceeded with url: /api/chuva/7217/dados (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: Hostname mismatch, certificate is not valid for 'cemaden.gov.br'. (_ssl.c:1032)")))


In [11]:
#1: Construir a Rota de Coleta de Dados
import requests
import pandas as pd
from datetime import datetime, timezone, timedelta

def coletar_dados_cemaden_francisco_morato():
    """
    Coleta os dados mais recentes dos pluviômetros do CEMADEN
    localizados em Francisco Morato-SP via API ArcGIS REST.
    """
    URL_BASE = "https://geoportal.sgb.gov.br/server/rest/services/Cemaden/MapServer/0/query"
    params = {
        'where': "cidade='FRANCISCO MORATO'",
        'outFields': 'codestacao,nome,chuva,dataHora,latitude,longitude',
        'f': 'json'
    }
    try:
        response = requests.get(URL_BASE, params=params, timeout=30)
        response.raise_for_status()  # Lança exceção se houver erro HTTP
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Erro ao realizar a requisição para o CEMADEN: {e}")
        return None


In [12]:
#2: Processamento da Resposta JSON
def processar_resposta_cemaden(json_data):
    """
    Processa o JSON retornado pela API do CEMADEN e o converte em um DataFrame do Pandas.
    """
    if not json_data or 'features' not in json_data or not json_data['features']:
        print("Nenhum dado de estação encontrado na resposta da API.")
        return pd.DataFrame()
    
    # Extrai os atributos de cada 'feature'
    records = [feature['attributes'] for feature in json_data['features']]
    df = pd.DataFrame(records)
    return df




In [13]:
def tratar_dataframe_cemaden(df):
    """
    Realiza o tratamento de dados no DataFrame do CEMADEN:
    - Converte a coluna de data/hora para o fuso local (America/Sao_Paulo).
    - Converte colunas numéricas para o tipo correto.
    - Renomeia colunas para maior clareza.
    """
    if df.empty:
        return df

    # Renomear colunas para um padrão mais limpo
    df = df.rename(columns={
        'codestacao': 'id_estacao',
        'nome': 'nome_estacao',
        'dataHora': 'data_hora_utc',
        'chuva': 'precipitacao_mm'
    })

    # Converter 'data_hora_utc' para datetime, tratando possíveis erros
    df['data_hora_utc'] = pd.to_datetime(df['data_hora_utc'], errors='coerce')

    # Definir o fuso horário de Brasília (UTC-3)
    fuso_horario_local = timezone(timedelta(hours=-3))

    # Converter para o fuso horário local
    df['data_hora_local'] = (
        df['data_hora_utc']
        .dt.tz_localize('UTC')
        .dt.tz_convert(fuso_horario_local)
    )

    # Converter colunas numéricas, substituindo valores inválidos por NaN
    numeric_cols = ['precipitacao_mm', 'latitude', 'longitude']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    return df


In [14]:
# Testando a pipeline completa
dados_json = coletar_dados_cemaden_francisco_morato()
df_raw = processar_resposta_cemaden(dados_json)
df_tratado = tratar_dataframe_cemaden(df_raw)

df_tratado.head()


Nenhum dado de estação encontrado na resposta da API.


""


In [30]:
import requests
import pandas as pd

url = "https://geoportal.sgb.gov.br/server/rest/services/Cemaden/MapServer/0/query"

params = {
    'where': "uf='AL'",
    'outFields': 'codestacao,nome,chuva,dataHora,latitude,longitude',
    'f': 'json'
}

resp = requests.get(url, params=params, verify=False)
data = resp.json()

if data.get("features"):
    records = [feat["attributes"] for feat in data["features"]]
    df = pd.DataFrame(records)
    print(df)
else:
    print("Nenhum dado encontrado para Pouso Alegre.")


     codestacao             nome  chuva               dataHora  latitude  \
0    270430217A  Chã da Jaqueira    1.0  2019-09-17 10:00:00.0 -9.621400   
1    270430217A  Chã da Jaqueira    1.2  2019-09-17 10:10:00.0 -9.621400   
2    270430217A  Chã da Jaqueira    0.2  2019-09-17 10:20:00.0 -9.621400   
3    270430217A  Chã da Jaqueira    0.2  2019-09-17 10:50:00.0 -9.621400   
4    270430217A  Chã da Jaqueira    0.2  2019-09-17 11:00:00.0 -9.621400   
..          ...              ...    ...                    ...       ...   
237  270860002A         Jatobá 2    0.6  2019-09-17 11:40:00.0 -9.799785   
238  270940001A           Centro    0.0  2019-09-17 09:20:00.0 -9.373764   
239  270940001A           Centro    0.0  2019-09-17 10:20:00.0 -9.373764   
240  270940001A           Centro    0.0  2019-09-17 11:20:00.0 -9.373764   
241  270940001A           Centro    0.0  2019-09-17 12:20:00.0 -9.373764   

     longitude  
0   -35.748000  
1   -35.748000  
2   -35.748000  
3   -35.748000  
4 

c:\Users\JoaoH\Documents\Apps_univesp\analise_climatica\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'geoportal.sgb.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,codestacao,nome,chuva,dataHora,latitude,longitude,cidade,uf
0,270430217A,Chã da Jaqueira,1.0,2019-09-17 10:00:00.0,-9.621400,-35.748000,MACEIÓ,AL
1,270430217A,Chã da Jaqueira,1.2,2019-09-17 10:10:00.0,-9.621400,-35.748000,MACEIÓ,AL
2,270430217A,Chã da Jaqueira,0.2,2019-09-17 10:20:00.0,-9.621400,-35.748000,MACEIÓ,AL
3,270430217A,Chã da Jaqueira,0.2,2019-09-17 10:50:00.0,-9.621400,-35.748000,MACEIÓ,AL
4,270430217A,Chã da Jaqueira,0.2,2019-09-17 11:00:00.0,-9.621400,-35.748000,MACEIÓ,AL
...,...,...,...,...,...,...,...,...
237,270860002A,Jatobá 2,0.6,2019-09-17 11:40:00.0,-9.799785,-36.104092,SÃO MIGUEL DOS CAMPOS,AL
238,270940001A,Centro,0.0,2019-09-17 09:20:00.0,-9.373764,-36.240199,VIÇOSA,AL
239,270940001A,Centro,0.0,2019-09-17 10:20:00.0,-9.373764,-36.240199,VIÇOSA,AL
240,270940001A,Centro,0.0,2019-09-17 11:20:00.0,-9.373764,-36.240199,VIÇOSA,AL
